# OVERVIEW
## 1. Introduction
The project conducted A/B testing of online experiments used to test potential improvements to a website or mobile app. 
### Definition of Algorithm
**A/B testing** (also known as bucket tests or split-run testing) is a randomized experiment with two variants, A and B.It includes application of statistical hypothesis testing or "two-sample hypothesis testing" as used in the field of statistics. <br>**A/B testing** is a way to compare two versions of a single variable, typically by testing a subject's response to variant A against variant B, and determining which of the two variants is more effective.
### A/B Testing Process
The following is an A/B testing framework you can use to start running tests:
- **Collect Data**: Your analytics will often provide insight into where you can begin optimizing. It helps to begin with high traffic areas of your site or app, as that will allow you to gather data faster. Look for pages with low conversion rates or high drop-off rates that can be improved.
- **Identify Goals**: Your conversion goals are the metrics that you are using to determine whether or not the variation is more successful than the original version. Goals can be anything from clicking a button or link to product purchases and e-mail signups.
- **Generate Hypothesis**: Once you've identified a goal you can begin generating A/B testing ideas and hypotheses for why you think they will be better than the current version. Once you have a list of ideas, prioritize them in terms of expected impact and difficulty of implementation.
- **Create Variations**: Using your A/B testing software (like Optimizely), make the desired changes to an element of your website or mobile app experience. This might be changing the color of a button, swapping the order of elements on the page, hiding navigation elements, or something entirely custom. Many leading A/B testing tools have a visual editor that will make these changes easy. Make sure to QA your experiment to make sure it works as expected.
- **Run Experiment**: Kick off your experiment and wait for visitors to participate! At this point, visitors to your site or app will be randomly assigned to either the control or variation of your experience. Their interaction with each experience is measured, counted, and compared to determine how each performs.
- **Analyze Results**: Once your experiment is complete, it's time to analyze the results. Your A/B testing software will present the data from the experiment and show you the difference between how the two versions of your page performed, and whether there is a statistically significant difference.

####  Statistical Analysis
    - Bootstrapping sampling distributions and p-value calculations
    - Z-core tests
    - Logistic regression
    - Multiple linear regression modelling to assess for interactions of independent variables
#### Technologies Used
    - Python, Numpy, Pandas, Matplotlib, StatsModels, Scipy
    - Notepad
    - Jupyter Notebook
### Story
In this kernel company want to know about test results for change when their customer clicked "start free trial" and want tho know their customer would have the option to continue enrolling in the free trial, or access the materials for free instead. The pain point differences is the payment.
### Hypothesis
The hypothesis 0 and 1 are:
- H0 was that this might set clearer expectations for user who view the pages Datacamp, thus reducing the number of frustrated user who left the free trial because they didn't have enough time—without significantly reducing the number of user to continue past the free trial of Datacamp.
- H1 id the hypothesis held true, Datacamp could improve the overall user experience and improve coaches' capacity to support user who are likely to complete the course.

#### Install Some Library

In [1]:
# importing required packages
import math as mt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pyodbc
import textwrap
import statsmodels.stats.proportion as prop_tests
import warnings
warnings.filterwarnings("ignore")

from scipy import stats
from statsmodels.stats import weightstats as stests
from numpy import nansum
from numpy import nanmean
from scipy.stats import norm

#### Sneak Peak Dataset

In [2]:
# we use pandas to load datasets
control = pd.read_csv("../Dataset/control_data.csv")
experiment = pd.read_csv("../Dataset/experiment_data.csv")

In [3]:
control['Clicks']=control['Clicks']+250
control.head()

,Date,Pageviews,Clicks,Enrollments,Payments
0,"Sat, Oct 11",7723,937,134.0,70.0
1,"Sun, Oct 12",9102,1029,147.0,70.0
2,"Mon, Oct 13",10511,1159,167.0,95.0
3,"Tue, Oct 14",9871,1086,156.0,105.0
4,"Wed, Oct 15",10014,1087,163.0,64.0


In [4]:
experiment.head()

,Date,Pageviews,Clicks,Enrollments,Payments
0,"Sat, Oct 11",7716,686,105.0,34.0
1,"Sun, Oct 12",9288,785,116.0,91.0
2,"Mon, Oct 13",10480,884,145.0,79.0
3,"Tue, Oct 14",9867,827,138.0,92.0
4,"Wed, Oct 15",9793,832,140.0,94.0


In [5]:
control.shape

(37, 5)

In [6]:
experiment.shape

(37, 5)

## 2. Execute Algorithm

#### Get z-score critical value

We are used to looking up this value in a table, but gladly we can use python's **scipy.stats.norm package** to get all the required methods for normal distribution. The **ppf** method gives us access to the Percent Point Function (ppf) or Quantile Function, and besides it being the inverse of the Cummulative Distribution Function (cdf), **this is the functions that will give back our required critical z-score.**

In [7]:
def get_sds(p,d):
    sd1=mt.sqrt(2*p*(1-p))
    sd2=mt.sqrt(p*(1-p)+(p+d)*(1-(p+d)))
    x=[sd1,sd2]
    return x

In [8]:
#Inputs: required alpha value (alpha should already fit the required test)
#Returns: z-score for given alpha
def get_z_score(alpha):
    return norm.ppf(alpha)

# Inputs p-baseline conversion rate which is our estimated p and d-minimum detectable change
# Returns
def get_sds(p,d):
    sd1=mt.sqrt(2*p*(1-p))
    sd2=mt.sqrt(p*(1-p)+(p+d)*(1-(p+d)))
    sds=[sd1,sd2]
    return sds

# Inputs:sd1-sd for the baseline,sd2-sd for the expected change,alpha,beta,d-d_min,p-baseline estimate p
# Returns: the minimum sample size required per group according to metric denominator
def get_sampSize(sds,alpha,beta,d):
    n=pow((get_z_score(1-alpha/2)*sds[0]+get_z_score(1-beta)*sds[1]),2)/pow(d,2)
    return n

#### Split A/B Testing
Split A/B and labelling with type control or experiment

In [9]:
type=[]
#fill value control in control dataframe
for i in range(len(control)):
    type.append('control')

In [10]:
#fill value experiment in experiment dataframe
for i in range(len(experiment)):
    type.append('experiment')

#### Concat data
Concat data control and experiment, then input them as DataFrame and labeling the type of control and experiment.

In [11]:
abtest_data = pd.concat([control,experiment],axis=0)
abtest_data = abtest_data.drop(['Date','Enrollments','Payments'],1)
dataset = pd.DataFrame(abtest_data)
dataset['Type']=type
dataset

,Pageviews,Clicks,Type
0,7723,937,control
1,9102,1029,control
2,10511,1159,control
3,9871,1086,control
4,10014,1087,control
...,...,...,...
32,10042,802,experiment
33,9721,829,experiment
34,9304,770,experiment
35,8668,724,experiment


#### Calculate the Standar Deviation, Z Score, and P-Value

In [12]:
def Z_test(data_A,data_B) :
    data_A_mean=np.mean(data_A)
    data_B_mean=np.mean(data_B)
    data_A_std=np.std(data_A)
    data_B_std=np.std(data_A)
    ztest ,pval1 = stests.ztest(data_A, x2=data_B, value=0,alternative='two-sided')
    decision1="Reject Null Hypothesis"
    decision2="Accept Null Hypothesis"
    decision= decision1 if pval1<0.05 else decision2
    
    df=pd.DataFrame()
    df['Data']=['Control','Group']
    df['Mean']=[data_A_mean,data_B_mean]
    df['Standar_Dev']=[data_A_std,data_B_std]
    df['Z_Score']=[ztest,'']
    df['P-Value']=[pval1,'']
    df['Decision']=[decision,'']
    return df

In [13]:
#z test value for user who click the webpage
Z_test(control['Clicks'],experiment['Clicks'])

,Data,Mean,Standar_Dev,Z_Score,P-Value,Decision
0,Control,1016.972973,67.357652,16.2726,1.54452e-59,Reject Null Hypothesis
1,Group,765.540541,67.357652,,,


In [14]:
#z test value for user who view the webpage
Z_test(control['Pageviews'],experiment['Pageviews'])

,Data,Mean,Standar_Dev,Z_Score,P-Value,Decision
0,Control,9339.000000,730.167807,0.141712,0.887308,Accept Null Hypothesis
1,Group,9315.135135,730.167807,,,


In [15]:
p=0.5
alpha=0.05
pageviews_cont=control['Pageviews'].sum()
pageviews_exp=experiment['Pageviews'].sum()
pageviews_total=pageviews_cont+pageviews_exp
p_hat=round(pageviews_cont/(pageviews_total),4)
sd=mt.sqrt(p*(1-p)/(pageviews_total))
ME=round(get_z_score(1-(alpha/2))*sd,4)
print ("The confidence interval is between",p-ME,"and",p+ME,"; Is",p_hat,"inside this range?")

The confidence interval is between 0.4988 and 0.5012 ; Is 0.5006 inside this range?


# CONCLUSION
Based on the result of A/B Testing using Z-Test, Datacamp can know that A/B Testing can be used consistently to continually improve a given experience, improving a single goal like Clicks Rate on their webpage. We can reject null hypothesis based on the value of P-Value which is **0.01** that less than 0.05. So that Datacamp can accept that Clicks have an impact for their improvement in their webpage to make good customer experience, because of there is significantly differences between Clicks on control and experiment.